In [7]:
import mediapipe as mp
import numpy as np
import cv2
import time

ModuleNotFoundError: No module named 'torch_geometric'

In [8]:
import torch 
import torch.nn as nn
import torch_geometric
import torch.optim as optimizer
device = torch.device('cuda:0')
input_size = 1
hidden_size, latent_size = 1, 1
print(device)

cuda:0


In [9]:
if torch.cuda.is_available():
    print("CUDA is available!")
    # CUDA 장치 이름 및 세부 정보 확인
    device_name = torch.cuda.get_device_name(0)
    print(f"Device: {device_name}")
else:
    print("CUDA is not available.")

CUDA is not available.


In [10]:
data = 'squat'
extension = 'mp4'

## 검은 화면에 그래프만 남기기 - 영상 전체

In [ ]:
# Mediapipe Holistic 초기화
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

# verbose 설정
verbose = False  # 상세 로그 출력 여부
data = "push"

# 비디오 파일 범위 처리 (운동_001.mp4부터 운동_100.mp4까지)
for i in range(16, 21):
    # 비디오 파일 경로 생성
    video_file = f'{data}/{data}_{i:03d}.{extension}'
    cap = cv2.VideoCapture(video_file)
    
    if not cap.isOpened():
        if verbose:
            print(f"Cannot open video {video_file}")
        continue
    
    
    # 영상 정보 가져오기
    w = round(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    h = round(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    # 출력 파일 경로
    out_file = f'data/{data}_processed/{data}_graph_{i:03d}.mp4'
    fourcc = cv2.VideoWriter_fourcc(*'DIVX')
    out = cv2.VideoWriter(out_file, fourcc, fps, (w, h))
    
    prev_time = 0
    
    # Mediapipe Holistic 실행
    with mp_holistic.Holistic(
        static_image_mode=True, min_detection_confidence=0.5, model_complexity=2) as holistic:
        while cap.isOpened():
            success, image = cap.read()
            if not success:
                if verbose:
                    print(f"End of video {video_file}")
                break
            
            curr_time = time.time()
            
            # 검은 화면 생성
            black_frame = np.zeros((h, w, 3), dtype=np.uint8)
            
            # BGR 이미지를 RGB로 변환 후 처리
            results = holistic.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
            
            # 랜드마크 그리기
            if results.left_hand_landmarks:
                mp_drawing.draw_landmarks(
                    black_frame, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
            if results.right_hand_landmarks:
                mp_drawing.draw_landmarks(
                    black_frame, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
            if results.face_landmarks:
                mp_drawing.draw_landmarks(
                    black_frame, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
                    landmark_drawing_spec=None,
                    connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_tesselation_style())
            if results.pose_landmarks:
                mp_drawing.draw_landmarks(
                    black_frame, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                    landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())
            
            # FPS 계산 및 표시
            sec = curr_time - prev_time
            prev_time = curr_time
            fps = 1 / (sec)
            fps_str = "FPS : %0.1f" % fps
            cv2.putText(black_frame, fps_str, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            
            # verbose가 True일 경우 FPS 출력
            if verbose:
                print(f"Processing frame {i} - FPS: {fps_str}")
            
            # 출력 영상 저장 (화면 출력은 없음)
            out.write(black_frame)
            
    cap.release()
    out.release()

cv2.destroyAllWindows()



OpenCV: Couldn't read video stream from file "push/push_016.mp4"
OpenCV: Couldn't read video stream from file "push/push_017.mp4"
OpenCV: Couldn't read video stream from file "push/push_018.mp4"
OpenCV: Couldn't read video stream from file "push/push_019.mp4"
OpenCV: Couldn't read video stream from file "push/push_020.mp4"


## 검은 화면에 그래프만 남기기 - 영상 초기 15프레임만

In [22]:
# 비디오 파일 범위 처리 (운동_001.mp4부터 운동_100.mp4까지)
for i in range(1, 101):
    # 비디오 파일 경로 생성
    video_file = f'{data}/{data}_{i:03d}.{extension}'
    cap = cv2.VideoCapture(video_file)
    
    if not cap.isOpened():
        print(f"Cannot open video {video_file}")
        continue
    
    # 영상 정보 가져오기
    w = round(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    h = round(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    # 출력 파일 경로
    out_file = f'data/{data}_start/{data}_graph_start_{i:03d}.mp4'
    fourcc = cv2.VideoWriter_fourcc(*'DIVX')
    out = cv2.VideoWriter(out_file, fourcc, fps, (w, h))
    
    prev_time = 0
    frame_count = 0  # 프레임 수 카운트
    
    # Mediapipe Holistic 실행
    with mp_holistic.Holistic(
        static_image_mode=True, min_detection_confidence=0.5, model_complexity=2) as holistic:
        while cap.isOpened() and frame_count < 15:  # 첫 30프레임만 처리
            success, image = cap.read()
            if not success:
                print(f"End of video {video_file}")
                break
            
            curr_time = time.time()
            
            # 검은 화면 생성
            black_frame = np.zeros((h, w, 3), dtype=np.uint8)
            
            # BGR 이미지를 RGB로 변환 후 처리
            results = holistic.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
            
            # 랜드마크 그리기
            if results.left_hand_landmarks:
                mp_drawing.draw_landmarks(
                    black_frame, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
            if results.right_hand_landmarks:
                mp_drawing.draw_landmarks(
                    black_frame, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
            if results.face_landmarks:
                mp_drawing.draw_landmarks(
                    black_frame, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
                    landmark_drawing_spec=None,
                    connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_tesselation_style())
            if results.pose_landmarks:
                mp_drawing.draw_landmarks(
                    black_frame, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                    landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())
            
            # FPS 계산 및 표시
            sec = curr_time - prev_time
            prev_time = curr_time
            fps = 1 / (sec)
            fps_str = "FPS : %0.1f" % fps
            cv2.putText(black_frame, fps_str, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            
            # 출력 영상 저장 (화면 출력은 없음)
            out.write(black_frame)
            
            # 프레임 카운트 증가
            frame_count += 1
    print(f'{i:03d} saved')    
    cap.release()
    out.release()

cv2.destroyAllWindows()
cv2.waitKey(1)

Cannot open video push/push_001.mp4
Cannot open video push/push_002.mp4
Cannot open video push/push_003.mp4
Cannot open video push/push_004.mp4
Cannot open video push/push_005.mp4
Cannot open video push/push_006.mp4
Cannot open video push/push_007.mp4
Cannot open video push/push_008.mp4
Cannot open video push/push_009.mp4
Cannot open video push/push_010.mp4
Cannot open video push/push_011.mp4
Cannot open video push/push_012.mp4
Cannot open video push/push_013.mp4
Cannot open video push/push_014.mp4
Cannot open video push/push_015.mp4
Cannot open video push/push_016.mp4
Cannot open video push/push_017.mp4
Cannot open video push/push_018.mp4
Cannot open video push/push_019.mp4
Cannot open video push/push_020.mp4
Cannot open video push/push_021.mp4
Cannot open video push/push_022.mp4
Cannot open video push/push_023.mp4
Cannot open video push/push_024.mp4
Cannot open video push/push_025.mp4
Cannot open video push/push_026.mp4
Cannot open video push/push_027.mp4
Cannot open video push/push_

OpenCV: Couldn't read video stream from file "push/push_001.mp4"
OpenCV: Couldn't read video stream from file "push/push_002.mp4"
OpenCV: Couldn't read video stream from file "push/push_003.mp4"
OpenCV: Couldn't read video stream from file "push/push_004.mp4"
OpenCV: Couldn't read video stream from file "push/push_005.mp4"
OpenCV: Couldn't read video stream from file "push/push_006.mp4"
OpenCV: Couldn't read video stream from file "push/push_007.mp4"
OpenCV: Couldn't read video stream from file "push/push_008.mp4"
OpenCV: Couldn't read video stream from file "push/push_009.mp4"
OpenCV: Couldn't read video stream from file "push/push_010.mp4"
OpenCV: Couldn't read video stream from file "push/push_011.mp4"
OpenCV: Couldn't read video stream from file "push/push_012.mp4"
OpenCV: Couldn't read video stream from file "push/push_013.mp4"
OpenCV: Couldn't read video stream from file "push/push_014.mp4"
OpenCV: Couldn't read video stream from file "push/push_015.mp4"
OpenCV: Couldn't read vid

-1

## 웹캠 영상 그래프로 변환

In [ ]:
# Mediapipe Pose 초기화
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(static_image_mode=False, 
                    model_complexity=1, 
                    enable_segmentation=False, 
                    min_detection_confidence=0.5, 
                    min_tracking_confidence=0.5)

# 비디오 입력 설정
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # BGR 이미지를 RGB로 변환
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Pose 추론
    results = pose.process(rgb_frame)
    
    # 검은 화면 생성
    black_frame = np.zeros((480, 640, 3), dtype=np.uint8)  # 480x640 해상도, 검은 배경
    
    # Pose 결과를 검은 화면에 그리기
    if results.pose_landmarks:
        mp_drawing.draw_landmarks(
            black_frame, 
            results.pose_landmarks, 
            mp_pose.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2),  # 랜드마크 스타일
            mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=2)  # 연결선 스타일
        )
    
    # 결과 화면 출력
    cv2.imshow('Pose Graph on Black Background', black_frame)
    cv2.imshow('video',frame)
    if cv2.waitKey(1) == ord('q'):  # ESC 키로 종료
        break

cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)